In [3]:
#import required packages
import pandas as pd

In [181]:
def metrics(date):
    orders = pd.read_excel(r'C:\Users\lauri\OneDrive\Documents\data\orders.xlsx')
    order_lines = pd.read_excel(r'C:\Users\lauri\OneDrive\Documents\data\order_lines.xlsx')
    commissions = pd.read_excel(r'C:\Users\lauri\OneDrive\Documents\data\commissions.xlsx')
    product_promotions = pd.read_excel(r'C:\Users\lauri\OneDrive\Documents\data\product_promotions.xlsx')
    
    #find dates for all orders and commissions
    times = pd.Series(orders['created_at'])
    times2 = pd.Series(commissions['date'])
    for i in range(len(times)):
        times[i] = times[i].strftime("%x")
    for i in range(len(times2)):
        times2[i] = times2[i].strftime("%x")
    
    #find orders and commissions which occured on date
    orders = orders[times == date]
    commissions = commissions[times2 == date]
    orders.reset_index(inplace=True)
    commissions.reset_index(inplace=True)
    
    #find the number of unique customers on date
    no_of_customers = len(orders['customer_id'].unique())
    
    #initialise counters
    no_of_items = 0
    total_discount = 0
    discount_rate = 0
    order_total = 0
    comms = [0,0,0,0,0]

    for i in range(len(orders)):
        ID = orders['id'][i]
        vendorID = orders['vendor_id'][i]
        
        #commission rate for the order
        rate = list(commissions[commissions['vendor_id'] == vendorID]['rate'])[0]
        
        #order lines for order i
        lines = order_lines[order_lines['order_id'] == ID]
        lines.reset_index(inplace=True)
        
        #update counters
        no_of_items = no_of_items + lines['quantity'].sum()
        difference = lines['full_price_amount'].sum() - lines['discounted_amount'].sum()
        total_discount = total_discount + difference
        discount_rate = discount_rate + sum(lines['quantity']*lines['discount_rate'])
        order_total = order_total + lines['total_amount'].sum()
        
        for j in range(len(lines)):
            #find corresponding promotion_id, if it exists
            prodID = lines['product_id'][j]
            times3 = pd.Series(product_promotions['date'])
            for k in range(len(times3)):
                times3[k] = times3[k].strftime("%x")
            pp = product_promotions[times3 == date]
            pp = list(pp[pp['product_id'] == prodID]['promotion_id'])
            if len(pp) > 0:
                promotionID = pp[0] #1,2,3,4 or 5
                
                #update commissions for relevant promotion
                comms[promotionID-1] = comms[promotionID-1] + rate*lines['total_amount'][j]
            
    av_discount = discount_rate/no_of_items
    av_order = order_total/len(orders)
    tot_comms = sum(comms)
    av_comms = tot_comms/len(orders)
    
    promos = pd.read_excel(r'C:\Users\lauri\OneDrive\Documents\data\promotions.xlsx')
    promos = promos['description']
    
    return {'customers': no_of_customers, 'total_discount_amount': total_discount, 'items': no_of_items, 'order_total_avg': av_order, 'discount_rate_avg': av_discount, 'commissions': {'promotions': {promos[0]: comms[0], promos[1]: comms[1], promos[2]: comms[2], promos[3]: comms[3], promos[4]: comms[4]}, 'total': tot_comms, 'order_average': av_comms}}

In [182]:
metrics('08/02/19')

{'customers': 10,
 'total_discount_amount': 20061245.64408114,
 'items': 3082,
 'order_total_avg': 16499829.576384336,
 'discount_rate_avg': 0.12032583559680812,
 'commissions': {'promotions': {'Google Ads': 819302.4,
   'Organic Google': 0,
   'Organic DuckDuckGo': 64855.079999999994,
   'DuckDuckGo Ads': 19995.20494732989,
   'Bing': 0},
  'total': 904152.6849473299,
  'order_average': 90415.268494733}}

## Test
We compare the actual data for 08/01/19 with the metrics output.

There are 9 orders from 9 unique customers. By inserting "print(prodID)" in the code when a promotion occurs, we can match the products and their promotions to the raw data. This gives promotions 835 (twice) and 272, which occur within orders 2-10 (the orders on 08/01/19) according to the raw data, as required. This strongly indicates that the function works.
This is an easily repeatable test.

Similarly, we can print out lists of customers, promotions or commission rates and check them against the raw data.
Observe that usually the number of customers is equal to the number of orders, but this may not always be true!

In [161]:
#Testing ground: where the code was originally written
orders = pd.read_excel(r'C:\Users\lauri\OneDrive\Documents\data\orders.xlsx')
order_lines = pd.read_excel(r'C:\Users\lauri\OneDrive\Documents\data\order_lines.xlsx')
commissions = pd.read_excel(r'C:\Users\lauri\OneDrive\Documents\data\commissions.xlsx')
product_promotions = pd.read_excel(r'C:\Users\lauri\OneDrive\Documents\data\product_promotions.xlsx')


#find dates for all orders
times = pd.Series(orders['created_at'])
for i in range(len(times)):
    times[i] = times[i].strftime("%x")
    
#find orders which occured on date
orders = orders[times == '08/01/19']
orders.reset_index(inplace=True)

ID = orders['id'][0]
lines = order_lines[order_lines['order_id'] == ID]
lines.reset_index(inplace=True)
times2 = pd.Series(product_promotions['date'])
for k in range(len(times2)):
    times2[k] = times2[k].strftime("%x")
pp=product_promotions[times2 == '08/01/19']
x = pp[pp['product_id'] == 572]['promotion_id']
#commissions = commissions[times2 == '08/01/19']
#y = commissions[commissions['vendor_id'] == 2]['rate']
pp

,date,product_id,promotion_id
0,2019-08-01,572,3
1,2019-08-01,242,4
2,2019-08-01,416,2
3,2019-08-01,272,2
4,2019-08-01,126,3
5,2019-08-01,486,3
6,2019-08-01,227,2
7,2019-08-01,139,4
8,2019-08-01,98,3
9,2019-08-01,835,5
